In [1]:
# 03_session_management.py
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

_STORE: dict[str, ChatMessageHistory] = {}

def get_history(session_id: str) -> ChatMessageHistory:
    if session_id not in _STORE:
        _STORE[session_id] = ChatMessageHistory()
    return _STORE[session_id]

def main():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant. Use history for the current session only."),
        MessagesPlaceholder("history"),
        ("user", "{input}")
    ])

    chat = RunnableWithMessageHistory(
        prompt | llm,
        get_history,
        input_messages_key="input",
        history_messages_key="history",
    )

    session_id = "default"

    print("Type `session:<name>` to switch sessions (e.g., session:alice). Type exit to quit.")
    while True:
        text = input(f"\n[{session_id}] You> ").strip()
        if not text:
            continue
        if text.lower() in {"exit", "quit"}:
            break

        if text.lower().startswith("session:"):
            session_id = text.split(":", 1)[1].strip() or "default"
            print(f"Switched to session: {session_id}")
            continue

        result = chat.invoke(
            {"input": text},
            config={"configurable": {"session_id": session_id}},
        )
        print(f"\n[{session_id}] Assistant:\n", result.content)

if __name__ == "__main__":
    main()

Type `session:<name>` to switch sessions (e.g., session:alice). Type exit to quit.
Switched to session: Anandh

[Anandh] Assistant:
 Nice to meet you, Anandh! How can I assist you today?

[Anandh] Assistant:
 Your name is Anandh.
Switched to session: Kumar

[Kumar] Assistant:
 I'm sorry, but I don't have access to your name or any personal information. How can I assist you today?
